In [ ]:
from sogym.env import sogym
train_env = sogym(nelx=100,nely=50,mode='train',observation_type='box_dense')

In [ ]:
from stable_baselines3.ppo import MlpPolicy
from imitation.algorithms.adversarial.airl import AIRL
from imitation.rewards.reward_nets import BasicShapedRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize, SubprocVecEnv

from imitation.util import logger as imit_logger

# Create the vectorized environment
num_cpu = 4 # Number of processes to use
venv= make_vec_env(lambda:train_env, n_envs=num_cpu,vec_env_cls=SubprocVecEnv)
#venv=VecNormalize(venv,gamma=1.0,norm_obs=False)

policy_kwargs = dict(
    net_arch=dict(pi=[512,512,512], vf=[512,512,512]),
)

learner = PPO("MlpPolicy",
    env=venv,
    n_steps=2048*4//num_cpu,  #def: 224
    #policy_kwargs=policy_kwargs,
    batch_size=2048*4, #def: 50*224
    ent_coef=3e-4,
    #gamma=1.0,
    learning_rate=3e-4,
    tensorboard_log="tb_logs",

)

custom_logger = imit_logger.configure(
        folder="test_logs",
        format_strs=["tensorboard", "stdout"],
    )

In [ ]:
reward_net = BasicShapedRewardNet(
    venv.observation_space, venv.action_space,
    #reward_hid_sizes=(32,32),
    #potential_hid_sizes=(32, 32),
    #discount_factor=1.0,
    #dropout_prob=0.5
)

#learner=PPO.load('imit',env=venv,device='cuda')

airl_trainer = AIRL(
    demonstrations=trajectories,
    demo_batch_size=2048,
    debug_use_ground_truth=True,
    init_tensorboard=True,
    log_dir="test_logs",
    n_disc_updates_per_round=1,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
    custom_logger= custom_logger

)

In [ ]:
airl_trainer.train(4000000)  # Note: set to 300000 for better results

In [ ]:
learner.save('imit',)

In [ ]:
env=sogym(mode='train',observation_type='box_dense')
env = make_vec_env(lambda:env, n_envs=1,vec_env_cls=SubprocVecEnv)
model=learner.load('imit')
#model.set_env(env)

In [ ]:
obs=env.reset()
dones=False

while dones== False:
    action, _states = model.predict(obs,deterministic=False)
    print(action)
    obs, rewards, dones, info = env.step(action)
  
print("Reward:",rewards[0])
fig=env.env_method('plot')[0]
fig